# Shaded Area Map
### Based on source from CFBImperialism

In [14]:
# Dependencies

import folium
import random
from folium.plugins import MarkerCluster
# from folium import Fullscreen
from folium import LayerControl
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
import json
import sys
import os
from PIL import Image
from geopy.distance import geodesic

# Path to .geojson file with State Boundries
geojson_path = os.path.join('..', 'data', 'vault', 'States_shapefile.shp')
# Load the states shapefile
gdf_states = gpd.read_file(geojson_path)

# Path to shapefile with all US counties
shapefile_path = os.path.join('..', 'data', 'vault', 'cb_2018_us_county_500k.shp')
gdf = gpd.read_file(shapefile_path)
# Set the initial CRS (assuming it's in EPSG:4326, but you may need to verify the original CRS)
gdf = gdf.set_crs(epsg=4326)
# # Transform to the desired CRS if needed
# gdf = gdf.to_crs(epsg=4326)

# Open School Info Table
school_info_path = os.path.join('..', 'data', 'arena_school_info.csv')
school_info = pd.read_csv(school_info_path)


## CHECK SHAPEFILES FOR COMPATIBILITY
# Set the CRS for both dataframes if it's missing
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # Assuming coordinates are in WGS 84 (lat/lon)

if gdf_states.crs is None:
    gdf_states.set_crs(epsg=4326, inplace=True)  # Assuming coordinates are in WGS 84 (lat/lon)

# school_info.head()

#### Create a Dictionary from the school_info dataframe
- dictionary is the default data shape the rest of the code requires

In [15]:


# Make sure hex1, hex2, hex3 are strings
school_info['hex1'] = school_info['hex1'].astype(str)
school_info['hex2'] = school_info['hex2'].astype(str)
school_info['hex3'] = school_info['hex3'].astype(str)

# Transform hex color codes to ensure they are valid - add leading 0s if necessary (6 digits)
def fix_hex_color(hex_color):
    if hex_color.startswith('#'):
        hex_color = hex_color[1:]  # Remove the leading '#'
    hex_color = hex_color.zfill(6)  # Pad with leading zeros to ensure 6 digits
    return f"#{hex_color[-6:]}"  # Return the last 6 characters

# Apply the function to the hex1 column
school_info['hex1'] = school_info['hex1'].apply(fix_hex_color)
# Apply to hex2 & hex3 as well
school_info['hex2'] = school_info['hex2'].apply(fix_hex_color)
school_info['hex3'] = school_info['hex3'].apply(fix_hex_color)

logo_dir = os.path.join('..', 'images', 'logos')


teams = {}
for index, row in school_info.iterrows():
    teams[row['Team']] = {
        'coords': (row['Longitude'], row['Latitude']),
        'color': row['hex1'],
        'logo': os.path.join(logo_dir, f"{row['logo_abv']}.png")
    }

    # Print a few entries to verify
    if index < 5:
        print(f"{row['Team']}: {teams[row['Team']]}")

#### HOTFIX -Changing primary color to hex2 for teams that border other teams with same color
### Change Omaha to hex2
teams['Omaha']['color'] = school_info.loc[school_info['Team'] == 'Omaha', 'hex2'].values[0]
## Arizona State to hex2
teams['Arizona State']['color'] = school_info.loc[school_info['Team'] == 'Arizona State', 'hex2'].values[0]
# print(teams)

# school_info.head()

Air Force: {'coords': (-104.8837269, 39.0137391), 'color': '#003087', 'logo': '..\\images\\logos\\afa.png'}
Alaska: {'coords': (-147.7638406, 64.84212435), 'color': '#236192', 'logo': '..\\images\\logos\\akf.png'}
Alaska Anchorage: {'coords': (-149.8727373, 61.20553644), 'color': '#00583d', 'logo': '..\\images\\logos\\aka.png'}
American Intl: {'coords': (-72.5543263, 42.1180027), 'color': '#000000', 'logo': '..\\images\\logos\\aic.png'}
Arizona State: {'coords': (-111.9108672, 33.4471565), 'color': '#8c1d40', 'logo': '..\\images\\logos\\asu.png'}


### Functions

In [16]:
#### V_2 #### TAKE ABOUT 4 TIMES LONGER THAN EUCLIDEAN DISTANCE
# Function to get the closest team using geopy's geodesic distance
def get_closest_team(lon, lat, teams):
    min_distance = float('inf')
    closest_team = None
    for team, info in teams.items():
        # Calculate the geodesic distance between the two points (lon, lat) and team's coordinates
        distance = geodesic((lat, lon), (info['coords'][1], info['coords'][0])).kilometers
        if distance < min_distance:
            min_distance = distance
            closest_team = team
    return closest_team

# Add a new column to the GeoDataFrame for the closest team using geopy's geodesic distance
gdf['closest_team'] = gdf.geometry.apply(lambda x: get_closest_team(x.centroid.x, x.centroid.y, teams))
gdf['color'] = gdf['closest_team'].apply(lambda x: teams[x]['color'])


### ORIGINAL CODE USING EUCLIDEAN DISTANCE ### FASTER
# def get_closest_team(lon, lat, teams):
#     min_distance = float('inf')
#     closest_team = None
#     for team, info in teams.items():
#         distance = ((lon - info['coords'][0]) ** 2 + (lat - info['coords'][1]) ** 2) ** 0.5
#         if distance < min_distance:
#             min_distance = distance
#             closest_team = team
#     return closest_team

# # Add a new column to the GeoDataFrame for the closest team
# gdf['closest_team'] = gdf.geometry.apply(lambda x: get_closest_team(x.centroid.x, x.centroid.y, teams))
# gdf['color'] = gdf['closest_team'].apply(lambda x: teams[x]['color'])


# Create a Folium map centered around Michigan
# m = folium.Map(location=[44.5, -85.5], zoom_start=6, tiles='https://stamen-tiles.a.ssl.fastly.net/watercolor/{z}/{x}/{y}.jpg',
#                attr='Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap contributors')

# m = folium.Map(location=[44.5, -85.5], zoom_start=6, tiles='https://stamen-tiles.a.ssl.fastly.net/terrain/{z}/{x}/{y}.jpg', 
#                attr='Map tiles by Stamen Design, CC BY 3.0 — Map data © OpenStreetMap contributors')

# m = folium.Map(location=[44.5, -85.5], zoom_start=6, tiles='https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png',
#                attr='© OpenStreetMap contributors © CARTO')

# 
m = folium.Map(location=[44.5, -85.5], zoom_start=6, tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png', 
               attr='© OpenStreetMap contributors © CARTO')
# m = folium.Map(location=[44.5, -85.5], zoom_start=6, 
#                 tiles="Stamen Toner Lite", attr='<a href="https://www.stadiamaps.com/" target="_blank">Stadia Maps</a>')




# Add counties or other geographical features on top
for _, row in gdf.iterrows():
    geo_json = {
        'type': 'Feature',
        'geometry': row['geometry'].__geo_interface__,  # Ensure it's in the correct format
        'properties': {
            'closest_team': row['closest_team'],  # Add properties like 'closest_team'
            'color': row['color']
        }
    }

    folium.GeoJson(
        geo_json,
        style_function=lambda feature: {
            'fillColor': feature['properties']['color'],  # Access color from properties
            'color': 'black',
            'weight': 0.4,
            'fillOpacity': 0.6,
            'zIndex': 3  # Ensure counties render above state boundaries
        },
        highlight_function=lambda x: {'weight': 3, 'color': 'blue'},  # Highlight on hover
        tooltip=folium.GeoJsonTooltip(fields=['closest_team'], aliases=['Closest Team:'])
    ).add_to(m)

    # Add state boundaries with a dark outline and higher zIndex
for _, row in gdf_states.iterrows():  # Assuming gdf_states contains the state geometries
    geo_json = {
        'type': 'Feature',
        'geometry': row['geometry'].__geo_interface__,  # Ensure it's in the correct format
        'properties': {}
    }

    folium.GeoJson(
        geo_json,
        style_function=lambda feature: {
            'fillColor': 'none',  # No fill for state polygons
            'color': 'red',  # Darker color for state borders
            'weight': 10,  # Thicker outline for better visibility
            'zIndex': 2,  # Ensure it renders above base layers but below counties
        }
    ).add_to(m)

######### SMARTER JITTERING - IDENTIFY PROBLEM PAIRS OF TEAMS AND ONLY APPLY TO THOSE
### ADJUSTMENTS
# Step Threshold
adjut_step = 0.01

from geopy.distance import geodesic

# Define a function to calculate geodesic distance between two coordinates
def calculate_distance(coords1, coords2):
    return geodesic(coords1, coords2).kilometers

# Define a function to apply a small offset to coordinates based on a direction
def offset_coordinates(lon, lat, step_size=0.15, direction="N"):
    if direction == "N":
        lat += step_size
    elif direction == "S":
        lat -= step_size
    elif direction == "E":
        lon += step_size
    elif direction == "W":
        lon -= step_size
    return lon, lat

# List of directions for systematic offsetting
directions = ["N", "S", "E", "W", "NE", "NW", "SE", "SW"]

# Step 1: Identify teams that are too close to each other
threshold_distance = 40  # Set your distance threshold in kilometers

# Store adjusted coordinates to avoid overwriting original positions
adjusted_coords = {}

# Compare each team with every other team to find close pairs
for idx1, (team1, info1) in enumerate(teams.items()):
    lat1, lon1 = info1['coords'][1], info1['coords'][0]

    # If this team has already been adjusted, skip it
    if team1 in adjusted_coords:
        continue

    # Compare against all other teams
    for idx2, (team2, info2) in enumerate(teams.items()):
        if idx1 == idx2:
            continue

        lat2, lon2 = info2['coords'][1], info2['coords'][0]
        distance = calculate_distance((lat1, lon1), (lat2, lon2))

        # If the distance is below the threshold, apply jitter/offset to the second team
        if distance < threshold_distance:
            if team1 not in adjusted_coords:
                adjusted_coords[team1] = (lon1, lat1)  # Keep the original coordinates

            direction = directions[idx2 % len(directions)]  # Cycle through directions to avoid overlap
            offset_lon, offset_lat = offset_coordinates(lon2, lat2, step_size=0.001, direction=direction)
            adjusted_coords[team2] = (offset_lon, offset_lat)  # Save adjusted coordinates for team2

# Step 2: Place the markers on the map
for team, info in teams.items():
    # Use adjusted coordinates if available, otherwise use original
    lon, lat = adjusted_coords.get(team, info['coords'])

    #     # Scaling icon size based on a zone factor
    zone_factor = info.get('zone_factor', 1)  # Default to 1 if no zone factor exists
    icon_size = int(50 * zone_factor)  # Adjust 50 to the base size you want

    # Add the marker with jittered or original logo location
    icon = folium.CustomIcon(icon_image=info['logo'], icon_size=(icon_size, icon_size))  # Adjust icon size as needed
    folium.Marker(
        location=[lat, lon],
        icon=icon,
        tooltip=team
    ).add_to(m)




## Add Random Jottering to team logos to avoid overlap
### VERSION 1

# # Function to jitter coordinates
# def jitter_coordinates(lon, lat, jitter_amount=0.01):
#     # Add a small random offset to both latitude and longitude
#     jittered_lon = lon + random.uniform(-jitter_amount, jitter_amount)
#     jittered_lat = lat + random.uniform(-jitter_amount, jitter_amount)
#     return jittered_lon, jittered_lat

# # Loop through your teams and apply jittering to the coordinates
# for team, info in teams.items():
#     lon, lat = info['coords']
#     jittered_lon, jittered_lat = jitter_coordinates(lon, lat)
#     ##Scaling icon size based on a zone factor
#     zone_factor = info.get('zone_factor', 1)  # Default to 1 if no zone factor exists
#     icon_size = int(50 * zone_factor)  # Adjust 50 to the base size you want

#     # Add a marker for the jittered logo location
#     icon = folium.CustomIcon(icon_image=info['logo'], icon_size=(icon_size, icon_size))  # Adjust icon size as needed
#     folium.Marker(
#         location=[jittered_lat, jittered_lon],
#         icon=icon,
#         tooltip=team
#     ).add_to(m)

## ORIGINAL CODE WITHOUT JITTERING
# for team, info in teams.items():
#     # Scaling icon size based on a zone factor
#     zone_factor = info.get('zone_factor', 1)  # Default to 1 if no zone factor exists
#     icon_size = int(50 * zone_factor)  # Adjust 50 to the base size you want
    
#     folium.Marker(
#         location=[info['coords'][1], info['coords'][0]],
#         icon=folium.CustomIcon(info['logo'], icon_size=(icon_size, icon_size))  # Scaled size
#     ).add_to(m)




## Other Add-ons and Options
# folium.plugins.Fullscreen().add_to(m) # add a fullscreen toggle
# folium.LayerControl().add_to(m) # Add layer control - allow user to switch base maps

# Save the map to an HTML file
output_file = os.path.join('..', 'TEMP', 'jittering_test_4.html')
m.save(output_file)


    

In [17]:
print(gdf.crs)
print(gdf_states.crs)


EPSG:4326
EPSG:4326
